In [2]:
# import data
import pandas as pd

df = pd.read_csv('../top_rated_wines.csv')

df = df[df['variety'].notna()]

data = df.to_dict('records')

In [3]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# create database vector
# Load the model to create embeddings
encoder = SentenceTransformer('all-MiniLM-L6-v2')

# create the vector database client
qdrant = QdrantClient(":memory:") # create in-memory Qdrant instance

# create vector config
vector_config = models.VectorParams(
    size = encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
    distance = models.Distance.COSINE
)

# create collection to store books
qdrant.recreate_collection(
    collection_name = 'top_wines',
    vectors_config = vector_config
)

# vectorize!
qdrant.upsert(
    collection_name='top_wines',
    points = [ models.PointStruct(id = idx,
                                  vector = encoder.encode(doc['notes']).tolist(), 
                                  payload = doc
                                 ) 
              for idx, doc in enumerate(data) ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [9]:
# Searching
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

hits = qdrant.search(
    collection_name = 'top_wines',
    query_vector = encoder.encode(user_prompt).tolist(),
    limit=3
)

for hit in hits:
    print(hit.payload,'score:',hit.score,'\n')

{'name': 'Catena Zapata Argentino Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 98.0, 'notes': '"The single-vineyard 2004 Malbec Argentino Vineyard spent 17 months in new French oak. Remarkably fragrant and complex aromatically, it offers up aromas of wood smoke, creosote, pepper, clove, black cherry, and blackberry. Made in a similar, elegant style, it is the most structured of the three single vineyard wines, needing a minimum of a decade of additional cellaring. It should easily prove to be a 25-40 year wine. It is an exceptional achievement in Malbec. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard of excellence for comparing all others. The brilliant, forward-thinking Nicolas Catena remains in charge, with his daughter, Laura, playing an increasingly large role. The Catena Zapata winery is an essential destination for fans of both architecture and wine in Mendoza. It is hard to believe, given the surge i

In [10]:
# define a variable to hold the searching results
search_results = [hit.payload for hit in hits]

In [14]:
import os
from openai import OpenAI

In [18]:
# load OPENAI_API_KEY
OPENAI_API_KEY_PATH = '/home/loc/Documents/OPENAI_API_KEY.txt'

with open(OPENAI_API_KEY_PATH) as f:
    OPENAI_API_KEY = f.read().strip()
# print(OPENAI_API_KEY)

In [19]:
# Now time to connect to the local large language model

# client = OpenAI(
#     base_url = 'http://127.0.0.1:8080/v1',# http://<your_api_server_ip>:port
#     api_key = "sk-no-key-required"
# )

client = OpenAI(
    api_key = OPENAI_API_KEY
)

completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role":"system","content":"You are chatbot, a wine specialist, Your top priority is to help the customer select wine"},
        {"role":"system","content":user_prompt},
        {"role":"system","content":str(search_results)}
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='I would recommend the Catena Zapata Argentino Vineyard Malbec 2004 from Argentina. This wine has a rating of 98.0 and is highly acclaimed for its complexity and elegance. It offers aromas of wood smoke, black cherry, blackberry, and more. It is structured and will benefit from additional cellaring, making it a wine that can age beautifully for 25-40 years. Catena Zapata is a renowned winery in Argentina, known for producing exceptional Malbec wines.', role='assistant', function_call=None, tool_calls=None)


In [20]:
completion

ChatCompletion(id='chatcmpl-9U3wNUuq3Y0zNsqFnQmPAQEP1R0Z4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I would recommend the Catena Zapata Argentino Vineyard Malbec 2004 from Argentina. This wine has a rating of 98.0 and is highly acclaimed for its complexity and elegance. It offers aromas of wood smoke, black cherry, blackberry, and more. It is structured and will benefit from additional cellaring, making it a wine that can age beautifully for 25-40 years. Catena Zapata is a renowned winery in Argentina, known for producing exceptional Malbec wines.', role='assistant', function_call=None, tool_calls=None))], created=1716952015, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=104, prompt_tokens=684, total_tokens=788))

In [21]:
completion.choices[0].message.content

'I would recommend the Catena Zapata Argentino Vineyard Malbec 2004 from Argentina. This wine has a rating of 98.0 and is highly acclaimed for its complexity and elegance. It offers aromas of wood smoke, black cherry, blackberry, and more. It is structured and will benefit from additional cellaring, making it a wine that can age beautifully for 25-40 years. Catena Zapata is a renowned winery in Argentina, known for producing exceptional Malbec wines.'